db명 : mydb
테이블명 : navermap

가게명 name
업종 major
주소 address
번호 tel
영업 open
시간 opentime
방문 visitrev
리뷰 blogrev
별점 star
링크 link

In [13]:
#필요한 모듈 불러오기~
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pymysql

#DB 연결하기
conn = pymysql.connect(host = '127.0.0.1', user ='root', password = 'mysql904#', db = 'mydb', charset= 'utf8')
print("DB가 연결되었습니다.")

#테이블 만들기
cur = conn.cursor()
cur.execute("create table navermap(\
num int,\
name varchar(50) not null,\
major varchar(20),\
address varchar(100) not null,\
tel varchar(15) not null,\
open varchar(10) not null,\
opentime varchar(50),\
visitrev int,\
blogrev int,\
star decimal(4,2),\
link varchar(100),\
primary key(num))")

DB가 연결되었습니다.


0

In [15]:
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get(f"https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

# 검색어 입력
keyword = input('검색어를 입력해주세요: \n')
search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)
time.sleep(2)

#먼저 html2로 들어가기
driver.switch_to.frame("searchIframe") 
time.sleep(2)

# html2에서 클릭할 가게 이름 찾아오기 
stores = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

j = 1 #카운터용
DATA =[]
print(f"\n총 {len(stores)}개를 검색하였습니다")

for i in range(len(stores)): #찾을 매장개수 만큼 클릭하게 설정
    stores[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  #매장 클릭
    time.sleep(2)

    driver.switch_to.default_content()  # 먼저 기존의 iframe에서 빠져나오기 html2에서 html1 으로 돌아가기
    driver.switch_to.frame("entryIframe") #매장을 이미 다 클릭했기에 html2,html3은 다 열려있음. html3 들어가기
    time.sleep(2)
    #html 내용 크롤링
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.replace("방문자 리뷰","").replace(",","").strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.replace("블로그 리뷰","").replace(",","").strip()

    try: # 별점이 없는 경우 있음
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "")
    except: # 별점 없는 경우 처리하기 위함​​
        별점 = 0
    try: # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
    except: # 링크 없는 경우 처리하기 위함
        링크 = ""

    time.sleep(2)
    cur.execute(f"insert into navermap values({j}, '{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', {방문}, {리뷰}, {별점}, '{링크}')")
    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)
    DATA.append([이름,업종,주소,번호,영업,시간,별점,방문,리뷰,링크])
    j += 1    
    driver.switch_to.default_content()  # html3에서 html1으로 돌아가기
    driver.switch_to.frame("searchIframe") # html1에서 html2로 다시 들어가기
    time.sleep(2)

cur.execute("UPDATE navermap SET star = NULL WHERE star = 0")
cur.execute("UPDATE navermap SET link = NULL WHERE link = ''")

conn.commit()
conn.close()
print("DB가 연결 해제되었습니다.")
driver.quit()

검색어를 입력해주세요: 
 이수역 맛집



총 10개를 검색하였습니다
1. 매장명: 정선콩국수 소소정
업종: 한식
주소: 서울 서초구 동광로 68 1층 정선콩국수 소소정
연락처: 0507-1433-8667
영업현황: 영업 중
영업시간: 15:00에 브레이크타임
별점: 0 / 616 / 207
링크: https://blog.naver.com/jin508815
2. 매장명: 지즐미육회관 사당점
업종: 소고기구이
주소: 서울 서초구 방배천로2길 39 1층
연락처: 0507-1492-3356
영업현황: 영업 중
영업시간: 21:00에 라스트오더
별점: 4.73 / 618 / 152
링크: https://blog.naver.com/hanafc3334
3. 매장명: 애플하우스
업종: 떡볶이
주소: 서울 동작구 동작대로27다길 29 2층
연락처: 02-595-1629
영업현황: 영업 중
영업시간: 20:30에 라스트오더
별점: 4.34 / 12513 / 8572
링크: 
4. 매장명: 외양간
업종: 육류,고기요리
주소: 서울 동작구 동작대로11길 16 1층
연락처: 0507-1484-8223
영업현황: 영업 전
영업시간: 15:00에 영업 시작
별점: 4.69 / 595 / 702
링크: https://www.instagram.com/cowhome_sadang/
5. 매장명: 팔각도 이수역점
업종: 닭갈비
주소: 서울 동작구 동작대로27다길 12 1층
연락처: 0507-1340-0420
영업현황: 영업 중
영업시간: 22:10에 라스트오더
별점: 4.72 / 4696 / 1312
링크: https://8gakdo.co.kr/
6. 매장명: 목구멍 이수점
업종: 육류,고기요리
주소: 서울 동작구 동작대로27길 16-4
연락처: 0507-1417-1959
영업현황: 영업 전
영업시간: 16:00에 영업 시작
별점: 0 / 288 / 2313
링크: https://www.moggumung.com/
7. 매장명: 이수가 본점
업종: 한식
주소: 서울 동작구 동작대로29가길 30 1층
연락처: 0507-1348-1372


In [5]:
#필요한 모듈 불러오기~
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pymysql

#DB 연결하기
conn = pymysql.connect(host = '127.0.0.1', user ='root', password = 'mysql904#', db = 'mydb', charset= 'utf8')
print("DB가 연결되었습니다.")

#테이블 만들기
cur = conn.cursor()
cur.execute("create table navermap2(\
num int,\
name varchar(50) not null,\
major varchar(20),\
address varchar(100) not null,\
tel varchar(15) not null,\
open varchar(10) not null,\
opentime varchar(50),\
visitrev int,\
blogrev int,\
star decimal(4,2),\
link varchar(100),\
primary key(num))")

DB가 연결되었습니다.


0

In [7]:
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get(f"https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

# 검색어 입력
keyword = input('검색어를 입력해주세요: \n')
search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)
time.sleep(2)

#먼저 html2로 들어가기
driver.switch_to.frame("searchIframe") 
time.sleep(2)

# html2에서 클릭할 가게 이름 찾아오기 
stores = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

j = 1 #카운터용
DATA =[]
print(f"\n총 {len(stores)}개를 검색하였습니다")

for i in range(len(stores)): #찾을 매장개수 만큼 클릭하게 설정
    stores[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  #매장 클릭
    time.sleep(2)

    driver.switch_to.default_content()  # 먼저 기존의 iframe에서 빠져나오기 html2에서 html1 으로 돌아가기
    driver.switch_to.frame("entryIframe") #매장을 이미 다 클릭했기에 html2,html3은 다 열려있음. html3 들어가기
    time.sleep(2)
    #html 내용 크롤링
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.replace("방문자 리뷰","").replace(",","").strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.replace("블로그 리뷰","").replace(",","").strip()

    try: # 별점이 없는 경우 있음
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "")
    except: # 별점 없는 경우 처리하기 위함​​
        별점 = 'Null'
    try: # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
        cur.execute(f"insert into navermap2 values({j}, '{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', {방문}, {리뷰}, {별점}, '{링크}')")
    except: # 링크 없는 경우 처리하기 위함
        링크 = 'Null'
        cur.execute(f"insert into navermap2 values({j}, '{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', {방문}, {리뷰}, {별점}, {링크})")
    
    time.sleep(2)
    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)
    DATA.append([이름,업종,주소,번호,영업,시간,별점,방문,리뷰,링크])
    j += 1    
    driver.switch_to.default_content()  # html3에서 html1으로 돌아가기
    driver.switch_to.frame("searchIframe") # html1에서 html2로 다시 들어가기
    time.sleep(2)


conn.commit()
conn.close()
print("DB가 연결 해제되었습니다.")
driver.quit()

검색어를 입력해주세요: 
 이수역 맛집



총 10개를 검색하였습니다
1. 매장명: 정선콩국수 소소정
업종: 한식
주소: 서울 서초구 동광로 68 1층 정선콩국수 소소정
연락처: 0507-1433-8667
영업현황: 영업 중
영업시간: 21:20에 라스트오더
별점: Null / 616 / 207
링크: https://blog.naver.com/jin508815
2. 매장명: 갯벌의조개 사당점
업종: 조개요리
주소: 서울 관악구 남현1길 68-6 1층 제나호
연락처: 0507-1420-2939
영업현황: 영업 중
영업시간: 22:30에 라스트오더
별점: Null / 266 / 1037
링크: Null
3. 매장명: 애플하우스
업종: 떡볶이
주소: 서울 동작구 동작대로27다길 29 2층
연락처: 02-595-1629
영업현황: 영업 중
영업시간: 20:30에 라스트오더
별점: 4.34 / 12513 / 8571
링크: Null
4. 매장명: 외양간
업종: 육류,고기요리
주소: 서울 동작구 동작대로11길 16 1층
연락처: 0507-1484-8223
영업현황: 영업 중
영업시간: 24:00에 영업 종료
별점: 4.69 / 595 / 702
링크: https://www.instagram.com/cowhome_sadang/
5. 매장명: 목구멍 이수점
업종: 육류,고기요리
주소: 서울 동작구 동작대로27길 16-4
연락처: 0507-1417-1959
영업현황: 영업 중
영업시간: 01:00에 영업 종료
별점: Null / 288 / 2313
링크: https://www.moggumung.com/
6. 매장명: 팔각도 이수역점
업종: 닭갈비
주소: 서울 동작구 동작대로27다길 12 1층
연락처: 0507-1340-0420
영업현황: 영업 중
영업시간: 22:10에 라스트오더
별점: 4.72 / 4697 / 1312
링크: https://8gakdo.co.kr/
7. 매장명: 이수가 본점
업종: 한식
주소: 서울 동작구 동작대로29가길 30 1층
연락처: 0507-1348-1372
영업현황: 영업 중
영업시간: 

In [1]:
#필요한 모듈 불러오기~
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pymysql

#DB 연결하기
conn = pymysql.connect(host = '127.0.0.1', user ='root', password = 'mysql904#', db = 'mydb', charset= 'utf8')
print("DB가 연결되었습니다.")

#테이블 만들기
cur = conn.cursor()
cur.execute("create table navermap3(\
num int,\
name varchar(50) not null,\
major varchar(20),\
address varchar(100) not null,\
tel varchar(15) not null,\
open varchar(10) not null,\
opentime varchar(50),\
visitrev int,\
blogrev int,\
star decimal(4,2),\
link varchar(100),\
primary key(num))")

DB가 연결되었습니다.


0

In [3]:
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get(f"https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

# 검색어 입력
keyword = input('검색어를 입력해주세요: \n')
search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)
time.sleep(2)

#먼저 html2로 들어가기
driver.switch_to.frame("searchIframe") 
time.sleep(2)

# html2에서 클릭할 가게 이름 찾아오기 
stores = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

j = 1 #카운터용
DATA =[]
print(f"\n총 {len(stores)}개를 검색하였습니다")

for i in range(len(stores)): #찾을 매장개수 만큼 클릭하게 설정
    stores[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  #매장 클릭
    time.sleep(2)

    driver.switch_to.default_content()  # 먼저 기존의 iframe에서 빠져나오기 html2에서 html1 으로 돌아가기
    driver.switch_to.frame("entryIframe") #매장을 이미 다 클릭했기에 html2,html3은 다 열려있음. html3 들어가기
    time.sleep(2)
    #html 내용 크롤링
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.replace("방문자 리뷰","").replace(",","").strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.replace("블로그 리뷰","").replace(",","").strip()

    try: # 별점이 없는 경우 있음
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "")
    except: # 별점 없는 경우 처리하기 위함​​
        별점 = None
    try: # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
    except: # 링크 없는 경우 처리하기 위함
        링크 = None
        
    time.sleep(2)
    cur.execute(f"""INSERT INTO navermap3 VALUES(
        {j},
        '{이름}',
        '{업종}', 
        '{주소}', 
        '{번호}', 
        '{영업}', 
        '{시간}', 
        '{방문}', 
        '{리뷰}',
        {'NULL' if 별점 is None else f"'{별점}'"},  # 수정된 부분
        {'NULL' if 링크 is None else f"'{링크}'"}  # 수정된 부분
        )
        """)
    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)
    DATA.append([이름,업종,주소,번호,영업,시간,별점,방문,리뷰,링크])
    j += 1    
    driver.switch_to.default_content()  # html3에서 html1으로 돌아가기
    driver.switch_to.frame("searchIframe") # html1에서 html2로 다시 들어가기
    time.sleep(2)


conn.commit()
conn.close()
print("DB가 연결 해제되었습니다.")
driver.quit()

검색어를 입력해주세요: 
 이수역 맛집



총 10개를 검색하였습니다
1. 매장명: 갯벌의조개 사당점
업종: 조개요리
주소: 서울 관악구 남현1길 68-6 1층 제나호
연락처: 0507-1420-2939
영업현황: 영업 중
영업시간: 22:30에 라스트오더
별점: None / 266 / 1037
링크: None
2. 매장명: 타파델디아
업종: 스페인음식
주소: 서울 서초구 방배천로 92 102동 103호
연락처: 0507-1321-7970
영업현황: 영업 중
영업시간: 21:00에 라스트오더
별점: None / 74 / 25
링크: https://catchtable.co.kr/tapadeldia
3. 매장명: 애플하우스
업종: 떡볶이
주소: 서울 동작구 동작대로27다길 29 2층
연락처: 02-595-1629
영업현황: 영업 중
영업시간: 20:30에 라스트오더
별점: 4.34 / 12513 / 8572
링크: None
4. 매장명: 외양간
업종: 육류,고기요리
주소: 서울 동작구 동작대로11길 16 1층
연락처: 0507-1484-8223
영업현황: 영업 중
영업시간: 24:00에 영업 종료
별점: 4.69 / 595 / 702
링크: https://www.instagram.com/cowhome_sadang/
5. 매장명: 목구멍 이수점
업종: 육류,고기요리
주소: 서울 동작구 동작대로27길 16-4
연락처: 0507-1417-1959
영업현황: 영업 중
영업시간: 01:00에 영업 종료
별점: None / 288 / 2313
링크: https://www.moggumung.com/
6. 매장명: 팔각도 이수역점
업종: 닭갈비
주소: 서울 동작구 동작대로27다길 12 1층
연락처: 0507-1340-0420
영업현황: 영업 중
영업시간: 22:10에 라스트오더
별점: 4.72 / 4697 / 1312
링크: https://8gakdo.co.kr/
7. 매장명: 이수가 본점
업종: 한식
주소: 서울 동작구 동작대로29가길 30 1층
연락처: 0507-1348-1372
영업현황: 영업 중
영업시간: 21